In [80]:
!pip install scikit-learn

# Checking if GPU is in use

In [81]:
pip show tensorflow


Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\l_alm\miniconda3\envs\pytorch_env\lib\site-packages
Requires: tensorflow-intel
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [82]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)
print("GPU Devices:", tf.config.list_physical_devices('GPU'))
from tensorflow.python.client import device_lib
print("Local Devices:", device_lib.list_local_devices())



TensorFlow Version: 2.12.0
GPU Devices: []
Local Devices: [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 114273222624118290
xla_global_id: -1
]


In [83]:
import tensorflow as tf
print(tf.__version__)


2.12.0


In [84]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)
print("cuDNN Version:", torch.backends.cudnn.version())
print("Number of GPUs:", torch.cuda.device_count())


PyTorch Version: 2.5.1
CUDA Available: True
CUDA Version: 12.1
cuDNN Version: 90100
Number of GPUs: 1


In [85]:
import torch
print(torch.cuda.is_available())


True


In [86]:
import torch
print(torch.cuda.get_device_name(0))


NVIDIA GeForce GTX 1660 Ti with Max-Q Design


In [87]:
import os
print("Current Working Directory:", os.getcwd())


Current Working Directory: C:\Users\l_alm\resnet1d-master


In [88]:

print(os.listdir())



['.gitattributes', '.gitignore', 'acnn1d.py', 'af-classification-from-a-short-single-lead-ecg-recording-the-physionet-computing-in-cardiology-challenge-2017-1.0.0', 'best_model.pth', 'challenge-2017', 'challenge2017.pkl', 'cnn1d.py', 'content', 'crnn1d.py', 'data', 'events.out.tfevents.1736883043.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736888460.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736888765.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736892255.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736924647.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736924821.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736925178.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736925442.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736926336.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736926387.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736926468.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736926618.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736926690.LAPTOP-8C1LKCLQ', 'events.out.tfevents.1736926738.LAPTOP-8C1LKCLQ', 'final_model_weights.pth', 'LICENS

In [89]:
!pip install tqdm


In [90]:
!pip install matplotlib


In [91]:
!pip install pandas


In [92]:
!pip install tensorboardX


In [93]:
!pip install torchsummary


# Adjusting to directory

In [94]:
import os
os.chdir(r'C:\Users\l_alm\resnet1d-master')  #adjust this to your base directory if needed
print(f"Current working directory: {os.getcwd()}")


Current working directory: C:\Users\l_alm\resnet1d-master


In [95]:
!pip install --upgrade torch torchvision


  Using cached torch-2.6.0-cp39-cp39-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp39-cp39-win_amd64.whl.metadata (6.3 kB)
Using cached torch-2.6.0-cp39-cp39-win_amd64.whl (204.1 MB)
Using cached torchvision-0.21.0-cp39-cp39-win_amd64.whl (1.6 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu121
    Uninstalling torch-2.5.1+cu121:
      Successfully uninstalled torch-2.5.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu121 requires torch==2.5.1+cu121, but you have torch 2.6.0 which is incompatible.


In [96]:
!pip install keras


In [97]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [98]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
print("Successfully imported pad_sequences from TensorFlow Keras")


Successfully imported pad_sequences from TensorFlow Keras


# checkng if there is enough memory for training

> Add blockquote



In [99]:
import psutil

memory = psutil.virtual_memory()
print(f"Available memory: {memory.available / 1e9:.2f} GB")
print(f"Total memory: {memory.total / 1e9:.2f} GB")


Available memory: 1.09 GB
Total memory: 16.94 GB


In [100]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import scipy.io
from matplotlib import pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm


def read_data_generated(n_samples, n_length, n_channel, n_classes, verbose=False):
    """
    Generated data

    This generated data contains one noise channel class, plus unlimited number of sine channel classes which are different on frequency.

    """
    all_X = []
    all_Y = []

    # noise channel class
    X_noise = np.random.rand(n_samples, n_channel, n_length)
    Y_noise = np.array([0]*n_samples)
    all_X.append(X_noise)
    all_Y.append(Y_noise)

    # sine channel classe
    x = np.arange(n_length)
    for i_class in range(n_classes-1):
        scale = 2**i_class
        offset_list = 2*np.pi*np.random.rand(n_samples)
        X_sin = []
        for i_sample in range(n_samples):
            tmp_x = []
            for i_channel in range(n_channel):
                tmp_x.append(np.sin(x/scale+2*np.pi*np.random.rand()))
            X_sin.append(tmp_x)
        X_sin = np.array(X_sin)
        Y_sin = np.array([i_class+1]*n_samples)
        all_X.append(X_sin)
        all_Y.append(Y_sin)

    # combine and shuffle
    all_X = np.concatenate(all_X)
    all_Y = np.concatenate(all_Y)
    shuffle_idx = np.random.permutation(all_Y.shape[0])
    all_X = all_X[shuffle_idx]
    all_Y = all_Y[shuffle_idx]

    # random pick some and plot
    if verbose:
        for _ in np.random.permutation(all_Y.shape[0])[:10]:
            fig = plt.figure()
            plt.plot(all_X[_,0,:])
            plt.title('Label: {0}'.format(all_Y[_]))

    return all_X, all_Y


#if __name__ == "__main__":
  #  read_data_physionet_2_clean_federated(m_clients=4)

# 1d resnet model definition

In [101]:
"""
resnet for 1-d signal data, pytorch version

Shenda Hong, Oct 2019
"""

import numpy as np
from collections import Counter
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return (torch.tensor(self.data[index], dtype=torch.float), torch.tensor(self.label[index], dtype=torch.long))

    def __len__(self):
        return len(self.data)

class MyConv1dPadSame(nn.Module):
    """
    extend nn.Conv1d to support SAME padding
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super(MyConv1dPadSame, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.conv = torch.nn.Conv1d(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel_size,
            stride=self.stride,
            groups=self.groups)

    def forward(self, x):

        net = x

        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)

        net = self.conv(net)

        return net

class MyMaxPool1dPadSame(nn.Module):
    """
    extend nn.MaxPool1d to support SAME padding
    """
    def __init__(self, kernel_size):
        super(MyMaxPool1dPadSame, self).__init__()
        self.kernel_size = kernel_size
        self.stride = 1
        self.max_pool = torch.nn.MaxPool1d(kernel_size=self.kernel_size)

    def forward(self, x):

        net = x

        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)

        net = self.max_pool(net)

        return net

class BasicBlock(nn.Module):
    """
    ResNet Basic Block
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups, downsample, use_bn, use_do, is_first_block=False):
        super(BasicBlock, self).__init__()

        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.stride = stride
        self.groups = groups
        self.downsample = downsample
        if self.downsample:
            self.stride = stride
        else:
            self.stride = 1
        self.is_first_block = is_first_block
        self.use_bn = use_bn
        self.use_do = use_do

        # the first conv
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.relu1 = nn.ReLU()
        self.do1 = nn.Dropout(p=0.5)
        self.conv1 = MyConv1dPadSame(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=self.stride,
            groups=self.groups)

        # the second conv
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.do2 = nn.Dropout(p=0.5)
        self.conv2 = MyConv1dPadSame(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=1,
            groups=self.groups)

        self.max_pool = MyMaxPool1dPadSame(kernel_size=self.stride)

    def forward(self, x):

        identity = x

        # the first conv
        out = x
        if not self.is_first_block:
            if self.use_bn:
                out = self.bn1(out)
            out = self.relu1(out)
            if self.use_do:
                out = self.do1(out)
        out = self.conv1(out)

        # the second conv
        if self.use_bn:
            out = self.bn2(out)
        out = self.relu2(out)
        if self.use_do:
            out = self.do2(out)
        out = self.conv2(out)

        # if downsample, also downsample identity
        if self.downsample:
            identity = self.max_pool(identity)

        # if expand channel, also pad zeros to identity
        if self.out_channels != self.in_channels:
            identity = identity.transpose(-1,-2)
            ch1 = (self.out_channels-self.in_channels)//2
            ch2 = self.out_channels-self.in_channels-ch1
            identity = F.pad(identity, (ch1, ch2), "constant", 0)
            identity = identity.transpose(-1,-2)

        # shortcut
        out += identity

        return out

class ResNet1D(nn.Module):
    """

    Input:
        X: (n_samples, n_channel, n_length)
        Y: (n_samples)

    Output:
        out: (n_samples)

    Pararmetes:
        in_channels: dim of input, the same as n_channel
        base_filters: number of filters in the first several Conv layer, it will double at every 4 layers
        kernel_size: width of kernel
        stride: stride of kernel moving
        groups: set larget to 1 as ResNeXt
        n_block: number of blocks
        n_classes: number of classes

    """

    def __init__(self, in_channels, base_filters, kernel_size, stride, groups, n_block, n_classes, downsample_gap=2, increasefilter_gap=4, use_bn=True, use_do=True, verbose=False):
        super(ResNet1D, self).__init__()

        self.verbose = verbose
        self.n_block = n_block
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.use_bn = use_bn
        self.use_do = use_do

        self.downsample_gap = downsample_gap # 2 for base model
        self.increasefilter_gap = increasefilter_gap # 4 for base model

        # first block
        self.first_block_conv = MyConv1dPadSame(in_channels=in_channels, out_channels=base_filters, kernel_size=self.kernel_size, stride=1)
        self.first_block_bn = nn.BatchNorm1d(base_filters)
        self.first_block_relu = nn.ReLU()
        out_channels = base_filters

        # residual blocks
        self.basicblock_list = nn.ModuleList()
        for i_block in range(self.n_block):
            # is_first_block
            if i_block == 0:
                is_first_block = True
            else:
                is_first_block = False
            # downsample at every self.downsample_gap blocks
            if i_block % self.downsample_gap == 1:
                downsample = True
            else:
                downsample = False
            # in_channels and out_channels
            if is_first_block:
                in_channels = base_filters
                out_channels = in_channels
            else:
                # increase filters at every self.increasefilter_gap blocks
                in_channels = int(base_filters*2**((i_block-1)//self.increasefilter_gap))
                if (i_block % self.increasefilter_gap == 0) and (i_block != 0):
                    out_channels = in_channels * 2
                else:
                    out_channels = in_channels

            tmp_block = BasicBlock(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=self.kernel_size,
                stride = self.stride,
                groups = self.groups,
                downsample=downsample,
                use_bn = self.use_bn,
                use_do = self.use_do,
                is_first_block=is_first_block)
            self.basicblock_list.append(tmp_block)

        # final prediction
        self.final_bn = nn.BatchNorm1d(out_channels)
        self.final_relu = nn.ReLU(inplace=True)
        # self.do = nn.Dropout(p=0.5)
        self.dense = nn.Linear(out_channels, n_classes)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        out = x

        # first conv
        if self.verbose:
            print('input shape', out.shape)
        out = self.first_block_conv(out)
        if self.verbose:
            print('after first conv', out.shape)
        if self.use_bn:
            out = self.first_block_bn(out)
        out = self.first_block_relu(out)

        # residual blocks, every block has two conv
        for i_block in range(self.n_block):
            net = self.basicblock_list[i_block]
            if self.verbose:
                print('i_block: {0}, in_channels: {1}, out_channels: {2}, downsample: {3}'.format(i_block, net.in_channels, net.out_channels, net.downsample))
            out = net(out)
            if self.verbose:
                print(out.shape)

        # final prediction
        if self.use_bn:
            out = self.final_bn(out)
        out = self.final_relu(out)
        out = out.mean(-1)
        if self.verbose:
            print('final pooling', out.shape)
        # out = self.do(out)
        out = self.dense(out)
        if self.verbose:
            print('dense', out.shape)
        # out = self.softmax(out)
        if self.verbose:
            print('softmax', out.shape)

        return out

In [102]:
"""
resnet for 1-d signal data, pytorch version

Shenda Hong, Oct 2019
"""

import numpy as np
from collections import Counter
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

class MyDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return (torch.tensor(self.data[index], dtype=torch.float), torch.tensor(self.label[index], dtype=torch.long))

    def __len__(self):
        return len(self.data)

class MyConv1dPadSame(nn.Module):
    """
    extend nn.Conv1d to support SAME padding
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups=1):
        super(MyConv1dPadSame, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.conv = torch.nn.Conv1d(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            kernel_size=self.kernel_size,
            stride=self.stride,
            groups=self.groups)

    def forward(self, x):

        net = x

        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)

        net = self.conv(net)

        return net

class MyMaxPool1dPadSame(nn.Module):
    """
    extend nn.MaxPool1d to support SAME padding
    """
    def __init__(self, kernel_size):
        super(MyMaxPool1dPadSame, self).__init__()
        self.kernel_size = kernel_size
        self.stride = 1
        self.max_pool = torch.nn.MaxPool1d(kernel_size=self.kernel_size)

    def forward(self, x):

        net = x

        # compute pad shape
        in_dim = net.shape[-1]
        out_dim = (in_dim + self.stride - 1) // self.stride
        p = max(0, (out_dim - 1) * self.stride + self.kernel_size - in_dim)
        pad_left = p // 2
        pad_right = p - pad_left
        net = F.pad(net, (pad_left, pad_right), "constant", 0)

        net = self.max_pool(net)

        return net

class BasicBlock(nn.Module):
    """
    ResNet Basic Block
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride, groups, downsample, use_bn, use_do, is_first_block=False):
        super(BasicBlock, self).__init__()

        self.in_channels = in_channels
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.stride = stride
        self.groups = groups
        self.downsample = downsample
        if self.downsample:
            self.stride = stride
        else:
            self.stride = 1
        self.is_first_block = is_first_block
        self.use_bn = use_bn
        self.use_do = use_do

        # the first conv
        self.bn1 = nn.BatchNorm1d(in_channels)
        self.relu1 = nn.ReLU()
        self.do1 = nn.Dropout(p=0.5)
        self.conv1 = MyConv1dPadSame(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=self.stride,
            groups=self.groups)

        # the second conv
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu2 = nn.ReLU()
        self.do2 = nn.Dropout(p=0.5)
        self.conv2 = MyConv1dPadSame(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=1,
            groups=self.groups)

        self.max_pool = MyMaxPool1dPadSame(kernel_size=self.stride)

    def forward(self, x):

        identity = x

        # the first conv
        out = x
        if not self.is_first_block:
            if self.use_bn:
                out = self.bn1(out)
            out = self.relu1(out)
            if self.use_do:
                out = self.do1(out)
        out = self.conv1(out)

        # the second conv
        if self.use_bn:
            out = self.bn2(out)
        out = self.relu2(out)
        if self.use_do:
            out = self.do2(out)
        out = self.conv2(out)

        # if downsample, also downsample identity
        if self.downsample:
            identity = self.max_pool(identity)

        # if expand channel, also pad zeros to identity
        if self.out_channels != self.in_channels:
            identity = identity.transpose(-1,-2)
            ch1 = (self.out_channels-self.in_channels)//2
            ch2 = self.out_channels-self.in_channels-ch1
            identity = F.pad(identity, (ch1, ch2), "constant", 0)
            identity = identity.transpose(-1,-2)

        # shortcut
        out += identity

        return out

class ResNet1D(nn.Module):
    """

    Input:
        X: (n_samples, n_channel, n_length)
        Y: (n_samples)

    Output:
        out: (n_samples)

    Pararmetes:
        in_channels: dim of input, the same as n_channel
        base_filters: number of filters in the first several Conv layer, it will double at every 4 layers
        kernel_size: width of kernel
        stride: stride of kernel moving
        groups: set larget to 1 as ResNeXt
        n_block: number of blocks
        n_classes: number of classes

    """

    def __init__(self, in_channels, base_filters, kernel_size, stride, groups, n_block, n_classes, downsample_gap=2, increasefilter_gap=4, use_bn=True, use_do=True, verbose=False):
        super(ResNet1D, self).__init__()

        self.verbose = verbose
        self.n_block = n_block
        self.kernel_size = kernel_size
        self.stride = stride
        self.groups = groups
        self.use_bn = use_bn
        self.use_do = use_do

        self.downsample_gap = downsample_gap # 2 for base model
        self.increasefilter_gap = increasefilter_gap # 4 for base model

        # first block
        self.first_block_conv = MyConv1dPadSame(in_channels=in_channels, out_channels=base_filters, kernel_size=self.kernel_size, stride=1)
        self.first_block_bn = nn.BatchNorm1d(base_filters)
        self.first_block_relu = nn.ReLU()
        out_channels = base_filters

        # residual blocks
        self.basicblock_list = nn.ModuleList()
        for i_block in range(self.n_block):
            # is_first_block
            if i_block == 0:
                is_first_block = True
            else:
                is_first_block = False
            # downsample at every self.downsample_gap blocks
            if i_block % self.downsample_gap == 1:
                downsample = True
            else:
                downsample = False
            # in_channels and out_channels
            if is_first_block:
                in_channels = base_filters
                out_channels = in_channels
            else:
                # increase filters at every self.increasefilter_gap blocks
                in_channels = int(base_filters*2**((i_block-1)//self.increasefilter_gap))
                if (i_block % self.increasefilter_gap == 0) and (i_block != 0):
                    out_channels = in_channels * 2
                else:
                    out_channels = in_channels

            tmp_block = BasicBlock(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=self.kernel_size,
                stride = self.stride,
                groups = self.groups,
                downsample=downsample,
                use_bn = self.use_bn,
                use_do = self.use_do,
                is_first_block=is_first_block)
            self.basicblock_list.append(tmp_block)

        # final prediction
        self.final_bn = nn.BatchNorm1d(out_channels)
        self.final_relu = nn.ReLU(inplace=True)
        # self.do = nn.Dropout(p=0.5)
        self.dense = nn.Linear(out_channels, n_classes)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        out = x

        # first conv
        if self.verbose:
            print('input shape', out.shape)
        out = self.first_block_conv(out)
        if self.verbose:
            print('after first conv', out.shape)
        if self.use_bn:
            out = self.first_block_bn(out)
        out = self.first_block_relu(out)

        # residual blocks, every block has two conv
        for i_block in range(self.n_block):
            net = self.basicblock_list[i_block]
            if self.verbose:
                print('i_block: {0}, in_channels: {1}, out_channels: {2}, downsample: {3}'.format(i_block, net.in_channels, net.out_channels, net.downsample))
            out = net(out)
            if self.verbose:
                print(out.shape)

        # final prediction
        if self.use_bn:
            out = self.final_bn(out)
        out = self.final_relu(out)
        out = out.mean(-1)
        if self.verbose:
            print('final pooling', out.shape)
        # out = self.do(out)
        out = self.dense(out)
        if self.verbose:
            print('dense', out.shape)
        # out = self.softmax(out)
        if self.verbose:
            print('softmax', out.shape)

        return out

In [103]:
import numpy as np
import pandas as pd
import scipy.io
from matplotlib import pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from collections import Counter
from tqdm import tqdm


def read_data_generated(n_samples, n_length, n_channel, n_classes, verbose=False):
    """
    Generated data

    This generated data contains one noise channel class, plus unlimited number of sine channel classes which are different on frequency.

    """
    all_X = []
    all_Y = []

    # noise channel class
    X_noise = np.random.rand(n_samples, n_channel, n_length)
    Y_noise = np.array([0]*n_samples)
    all_X.append(X_noise)
    all_Y.append(Y_noise)

    # sine channel classe
    x = np.arange(n_length)
    for i_class in range(n_classes-1):
        scale = 2**i_class
        offset_list = 2*np.pi*np.random.rand(n_samples)
        X_sin = []
        for i_sample in range(n_samples):
            tmp_x = []
            for i_channel in range(n_channel):
                tmp_x.append(np.sin(x/scale+2*np.pi*np.random.rand()))
            X_sin.append(tmp_x)
        X_sin = np.array(X_sin)
        Y_sin = np.array([i_class+1]*n_samples)
        all_X.append(X_sin)
        all_Y.append(Y_sin)

    # combine and shuffle
    all_X = np.concatenate(all_X)
    all_Y = np.concatenate(all_Y)
    shuffle_idx = np.random.permutation(all_Y.shape[0])
    all_X = all_X[shuffle_idx]
    all_Y = all_Y[shuffle_idx]

    # random pick some and plot
    if verbose:
        for _ in np.random.permutation(all_Y.shape[0])[:10]:
            fig = plt.figure()
            plt.plot(all_X[_,0,:])
            plt.title('Label: {0}'.format(all_Y[_]))

    return all_X, all_Y


#if __name__ == "__main__":
  #  read_data_physionet_2_clean_federated(m_clients=4)

In [104]:
!pip install tensorboardX

# Preprocessing function has scaling and encoding..

In [105]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
def preprocess_data(file_path):
    # Load dataset
    df = pd.read_csv(file_path)

    # Handle missing values (drop rows with missing data)
    df = df.dropna()

    # Encode categorical columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    # Separate features and labels
    X = df.drop("target", axis=1)
    y = df["target"]

    # Normalize numerical features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split into train and test sets
    n_samples , n_features = X.shape
    n_channel = n_features
    n_length = 1

    X = X.reshape(n_samples, n_channel, n_length)



    return X ,y

In [106]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE

def preprocess_data_with_smote(file_path):
    """
    Preprocess the input CSV data for ResNet1D with SMOTE.

    Args:
        file_path (str): Path to the CSV file.

    Returns:
        tuple: Processed data (X), labels (y).
    """
    # Load dataset
    df = pd.read_csv(file_path)

    # Handle missing values (drop rows with missing data)
    df = df.dropna()

    # Encode categorical columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    # Separate features and labels
    X = df.drop("target", axis=1).values
    y = df["target"].values

    # Normalize numerical features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Apply SMOTE to balance the classes
    smote = SMOTE(random_state=42)
    X, y = smote.fit_resample(X, y)

    # Reshape data for ResNet1D
    n_samples, n_features = X.shape
    n_channel = n_features
    n_length = 1

    X = X.reshape(n_samples, n_channel, n_length)

    return X, y



In [107]:
!pip install imbalanced-learn


# Model Training and evaluating

In [108]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Found existing installation: torch 2.6.0
Uninstalling torch-2.6.0:
  Successfully uninstalled torch-2.6.0
Found existing installation: torchvision 0.21.0
Uninstalling torchvision-0.21.0:
  Successfully uninstalled torchvision-0.21.0
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp39-cp39-win_amd64.whl (2449.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp39-cp39-win_amd64.whl (6.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp39-cp39-win_amd64.whl (4.1 MB)


In [114]:
!pip install torchinfo


In [134]:
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
# Assuming the preprocess_data, MyDataset, and ResNet1D definitions are correct and loaded above this code

# Load and preprocess data
#data, label = preprocess_data_with_smote(r'C:\Users\l_alm\resnet1d-master\content\customerTargeting.csv')
data, label = preprocess_data(r'C:\Users\l_alm\resnet1d-master\content\customerTargeting.csv')
print(data.shape, Counter(label))
dataset = MyDataset(data, label)

#split dataset and prepare DataLoaders
#calculate split sizes 60,20,20
total_size = len(dataset)
train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size  #checking
# Print split sizes
print(f"Train size: {train_size}, Validation size: {val_size}, Test size: {test_size}")
# split dataset
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)
print(f"Train batches: {len(train_loader)}, Validation batches: {len(val_loader)}, Test batches: {len(test_loader)}")
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_classes = 3

model = ResNet1D(
    in_channels=70, #NO of features
    base_filters=64,#on the authors github repository he added a comment that 64 is for resnet1d
    kernel_size=16, 
    stride=2,
    n_block=48,
    groups=32,
    n_classes=n_classes,
    downsample_gap=6,#downsampling happens at every block ->  increases the efficiency of feature compression
    increasefilter_gap=12,
    verbose=False
)




model.to(device)

#compute FLOPs before training
from torchinfo import summary
summary(model, input_size=(1, 70, 128))  

#optimizer loss function and scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)  # Lower learning rate
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
class_weights = compute_class_weight('balanced', classes=np.unique(label), y=label)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

loss_func = torch.nn.CrossEntropyLoss(weight=class_weights)

#early stopping parameters
early_stopping_patience = 20
best_val_loss = float('inf')
patience_counter = 0

#training settings
n_epoch = 200

# Training and validation loop
for epoch in range(n_epoch):
    # Training
    model.train()
    train_loss = 0
    all_train_labels = []
    all_train_preds = []

    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}", leave=False):
        input_x, input_y = tuple(t.to(device) for t in batch)
        pred = model(input_x)
        loss = loss_func(pred, input_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        all_train_labels.extend(input_y.cpu().numpy())
        all_train_preds.extend(pred.argmax(dim=1).cpu().numpy())

    #compute training loss and F1 score
    train_f1 = f1_score(all_train_labels, all_train_preds, average='weighted')
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Train F1 = {train_f1:.4f}")

    # Validation
    model.eval()
    val_loss = 0
    all_val_labels = []
    all_val_preds = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}", leave=False): 
            input_x, input_y = tuple(t.to(device) for t in batch)
            pred = model(input_x)
            loss = loss_func(pred, input_y)
            
            val_loss += loss.item()
            all_val_labels.extend(input_y.cpu().numpy())
            all_val_preds.extend(pred.argmax(dim=1).cpu().numpy())

    #compute validation loss and F1 score
    val_f1 = f1_score(all_val_labels, all_val_preds, average='weighted')
    print(f"Epoch {epoch+1}: Val Loss = {val_loss:.4f}, Val F1 = {val_f1:.4f}")
    
    #update lR scheduler
    scheduler.step(val_loss)

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")  #save best model
    else:
        patience_counter += 1

    if patience_counter >= early_stopping_patience:
        print("Early stopping triggered!")
        break



#final evaluation
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
all_pred_prob = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Final Testing", leave=False):
           input_x, input_y = tuple(t.to(device) for t in batch)
           pred = model(input_x)
           all_pred_prob.append(pred.cpu().data.numpy())
all_pred_prob = np.concatenate(all_pred_prob)
all_pred = np.argmax(all_pred_prob, axis=1)

all_test_labels = []
for _, labels in test_loader:
    all_test_labels.extend(labels.cpu().numpy())  # Fix here


all_test_labels = np.array(all_test_labels)
print("Test Labels:", all_test_labels)
print(classification_report(all_test_labels, all_pred))  #fix argument order
print('Training complete.')


(6620, 70, 1) Counter({1: 3076, 2: 1877, 0: 1667})
Train size: 3972, Validation size: 1324, Test size: 1324
Train batches: 63, Validation batches: 21, Test batches: 21


Epoch 1: Train Loss = 72.5710, Train F1 = 0.3411


Epoch 1: Val Loss = 22.8931, Val F1 = 0.3769


Epoch 2: Train Loss = 70.5959, Train F1 = 0.3816


Epoch 2: Val Loss = 22.7125, Val F1 = 0.4271


...

Epoch 74: Val Loss = 20.6745, Val F1 = 0.5455


Epoch 75: Train Loss = 60.2592, Train F1 = 0.5525


C:\Users\l_alm\AppData\Local\Temp\ipykernel_40604\4028041525.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


Epoch 75: Val Loss = 20.5998, Val F1 = 0.5416
Early stopping triggered!


Test Labels: [2 2 1 ... 1 0 1]
              precision    recall  f1-score   support

           0       0.32      0.25      0.28       354
           1       0.66      0.68      0.67       618
           2       0.51      0.61      0.56       352

    accuracy                           0.55      1324
   macro avg       0.50      0.51      0.50      1324
weighted avg       0.53      0.55      0.54      1324

Training complete.
